### Programming Task No. 3
### Name: Jeremy Marcus Tan
### ID No: 204947



In [1]:
import numpy as np

In [2]:
def positivedefinite(M):

    '''
    We use the test for positive definiteness theorem 1 in the slides.
    Given a symmetric matrix M, it is positive definite if and only if 
    it is regular and has all positive pivots.

    This function performs a simplified version of Gaussian elimination 
    without row swaps, and checks that all pivot elements are positive.
    '''
    ispositivedefinite = True

    # Matrix dimension, we are only dealing with square matrices
    n = M.shape[0]
    
    M = M.copy() 
    
    # Perform Gaussian elimination using only type three row operations
    for i in range(n):
        for j in range(i + 1, n):
             # A zero pivot indicates that a row swap is needed; thus the matrix is not regular.
            if M[i, i] == 0:
                ispositivedefinite = False
                # Stop the function if pivot is zero
                return ispositivedefinite  
            M[j] = (-M[j, i]/M[i, i])*M[i] + M[j]
    
    # After elimination, identify pivot elements (first nonzero entry in each row)
    pivot_col = -1
    pivot_values = []

    for row in M:
        found = False
        for col_idx, val in enumerate(row):
            # We look at the first non-zero value in the column to the right of previous pivot
            if val != 0 and col_idx > pivot_col:
                pivot_values.append(val)
                pivot_col = col_idx
                found = True
                break  # Only take the first valid pivot in this row
        if not found:
            # If no pivot found in this row, then the matrix is not regular.
            ispositivedefinite = False
            return ispositivedefinite

    # Last, we check if all pivot are positive. 
    
    if all(val > 0 for val in pivot_values):
        ispositivedefinite = True
    else:
        ispositivedefinite = False
        
    return ispositivedefinite

In [3]:
def quadratic_coeffs_to_matrix(coeffs):

    # First, we solve for n so that we set the dimensions of the matrix.

    '''
    First let L = length of the coeffs array.
    Quadratic/Cross Product Terms: n(n+1)/2 terms
    Linear Terms: n terms
    Constant Term: 1 term

    Thus, L = n(n+1)/2 + n + 1
    So, we have to find the roots of n^2 + 3n + 2 - 2L = 0
    '''

    a = 1
    b = 3
    c = 2 - 2*len(coeffs)

    # We use the quadratic formula to solve.
    # Since n can't be negative, choose only the positive root.
    n = ((-b) + np.sqrt(b**2 - 4*a*c))/2*a
    
    # set n as an integer
    n = int(n)
    A = np.zeros((n, n))

    # Keeps track of cross term coefficients in the coeffs list
    counter = 0 

    # Fill the matrix, row by row
    for i in range(n):
        # Since A is symmetric, A_ji = A_ij
        # Thus, we can just fill the upper triangle of A (including diagonal),
        # and copy to the lower triangle
        for j in range(i,n):
            if i == j:
                # The diagonal entries are two times the coefficient of the 
                # quadratic terms x^2, y^2, ...
                A[i,i] = 2* coeffs[i]
            else:
                # 
                A[i,j] = coeffs[n+counter]
                A[j,i] = coeffs[n+counter]
                counter += 1

    # Linear terms
    b = np.zeros((n, 1))

    # Coefficients of linear terms are stored at the end before the constant
    # We fill b from bottom to top to match variable ordering
    for idx, i in enumerate(range(-2, -2 - n, -1)):
        b[n-1-idx, 0] = coeffs[i] 
    

    # Constant term is the last element of coeffs
    c = coeffs[-1]

    return A, b, c

In [4]:
def backSubstitution(U, b):
    """
    U = n x n upper triangular matrix with non-zero diagonal entries
    b = n x 1 vector
    """
    n = len(b)
    x = np.zeros(n)
    x[n - 1] = b[n - 1]/U[n - 1, n - 1]

    for i in range(n - 2, -1, -1):
        SUM = sum([U[i, j]*x[j] for j in range(i, n)])
        x[i] = (b[i] - SUM)/U[i, i]
    
    return x
def rowEchelonForm_NoType1(M):
    """
    M = m x n matrix with non-zero pivots
    """
    m, n = np.shape(M)
    minmn = min(m, n)
    M = M.copy() # To avoid changing the original matrix
    
    for i in range(minmn):
        for j in range(i + 1, m):
            M[j] = (-M[j, i]/M[i, i])*M[i] + M[j]
    
    return M
    
def solveLinearSystemGaussian(M, b):
    """
    M = n x n regular matrix
    b = n x 1 vector of constant terms
    """
    n = np.shape(M)[0]
    Mb = np.hstack((M, b))
    Uc = rowEchelonForm_NoType1(Mb)

    return backSubstitution(Uc[:, : n], Uc[:, -1])

In [5]:
def quadratic_minimization(coefficients):
    A,b,c = quadratic_coeffs_to_matrix(coefficients)

    # Check if A is positive definite
    if positivedefinite(A) == False:
        return False

    else:
        # We solve for A^(-1)b using Gaussian Elimination followed by back substitution
        # The output of the function is reshaped into a column vector
        A_inverse_b = solveLinearSystemGaussian(A, b).reshape(-1,1)

        # Solve for the unique minimizer
        min_point = - A_inverse_b

        # Solve for the minimum value of the function
        min_value = (-1/2) * (b.T @ A_inverse_b) + c

        return min_point, min_value
        

In [6]:
# Test: Not Positive Definite
'''
 For this example, 

 A = [[1,2,3],
      [2,3,7],
      [3,7,8]]

When doing Gaussian elimination, one of the pivots is negative, 
which means that A is not positive definite.
'''
quadratic_minimization(np.array([0.5,1.5,4,2,3,7,2,4,3,4], dtype = 'float64'))




False

In [7]:
rowEchelonForm_NoType1(np.array([[1,2,3], [2,3,7], [3,7,8]], dtype = 'float64'))

array([[ 1.,  2.,  3.],
       [ 0., -1.,  1.],
       [ 0.,  0.,  0.]])

In [8]:
# Test: Positive Definite
'''
 For this example, 

 A = [[10,5,2],
      [5,3,2],
      [2,2,3]]

When doing Gaussian elimination with only type three row operations, 
we get an upper triangular matrix with only positive pivots. This means that
A is a positive definite matrix. 
'''
quadratic_minimization(np.array([5,1.5,1.5,5,2,2,2,3,4,6], dtype = 'float64'))




(array([[ 2.33333333],
        [-5.33333333],
        [ 0.66666667]]),
 array([[1.66666667]]))

In [9]:
rowEchelonForm_NoType1(np.array([[10,5,2], [5,3,2], [2,2,3]], dtype = 'float64'))

array([[10. ,  5. ,  2. ],
       [ 0. ,  0.5,  1. ],
       [ 0. ,  0. ,  0.6]])

In [10]:
# Class Example
quadratic_minimization(np.array([1,2,2,2,1,1,0,6,-7,5], dtype = 'float64'))


(array([[ 2.],
        [-3.],
        [ 2.]]),
 array([[-11.]]))

In [11]:
# Class Example
quadratic_minimization(np.array([4,3,-2,3,-2,1], dtype = 'float64'))



(array([[-0.31818182],
        [ 0.22727273]]),
 array([[0.29545455]]))